In [14]:
"""
属性描述符, 实现：
__get__ 方法
__set__ 方法
__delete__ 方法
"""
import numbers


class IntField:
    """
    实现 __set___ 和 __get__ 叫做数据描述符
    """
    def __get__(self, instance, owner):
        """
        在获取属性赋值的时候，会调用 __get__ 方法
        """
        return self.value
    
    def __set__(self, instance, value):
        """
        在对属性赋值的时候，会调用 __set__ 方法
        
        instance 是使用属性描述符的类的实例对象
        应该把 value 存储在 IntField 的实例对象中
        而不是 instance.age = value 
        因为只要对 instance.age 赋值，就会调用 __set___ 方法
        这样就形成了死循环
        应该是 self.value = value
        """
        if not isinstance(value, numbers.Integral):
            raise ValueError("int value need")
        self.value = value
    
    def __delete__(self, instance):
        pass
    
    
class NonDataField:
    """
    只实现 __get__ 叫做 非数据描述符
    
    和数据描述符相比，区别不仅仅是只实现了 __get__ 的区别
    他们两者的 属性查找过程或顺序 也是不同的
    """
    def __get__(self, instance, owner):
        return "NonDataField"

"""
如果 user 是某个类的实例，那么 user.age (以及等价的getattr(user, 'age'))
首先调用 __getattribute__, 如果类定义了 __getattr__ 方法，
那么在 __getattribute__ 抛出 AttributeError 异常的时候会调用 __getattr__,
对于描述符 __get__ 的调用，则是发生在 __getattribute__ 的内部
user = User(), 那么 user.age 顺序如下：


我的理解：以下逻辑其实是 __getattribute__ 中写好的默认逻辑
所以一般情况下不要覆盖 __getattribute__ 方法 ==>
1 和 2 的情况都是对于 age 是类变量的情况而言 
==> 分为了 age 是 数据描述符 和 非数据描述符 的两种情况

1. 如果 age 是出现在 User (age 是类变量) 或者 User 的基类 __dict__ 中，
    并且 age 是数据描述符(data descriptor), 不论实例对象中是否有同名的属性
    都是：
    在获取属性值的时候 调用数据描述符的 __get__ 方法
    在对属性赋值的时候 调用数据描述符的 __set__ 方法
    
2. 如果 age 出现在 User 或基类的 __dict__ 中, 非数据描述符只实现了 __get__ 方法
    01. 如果 age 是非数据描述符(non-data descriptor)，不论 实例对象中是否有同名属性 都：调用 __get__ 方法
    02. 否则返回 __dict__['age']
    03. 如果对属性赋值，那么给 User.age 会使得 User.age 指向新的值，从而 User.age 也变成了普通的变量
    04. 如果对属性取值，那么取 User.age 会使调用 User.age 的 __get__ 方法，
        如果取 user.age 不论是否 user 有同名 age 属性，都会调用 User.age 的 __get__ 方法
    
3. 如果 age 出现在 User 的实例对象 user 的 __dict__ 中，那么直接返回 user.__dict__['age']
    若在实例对象中使用属性描述符，则该变量不是属性描述符对象，而仅仅是一个普通的变量
    user.__dict__['age'] 指向的不是 数据描述符对象 而是一个具体的值, 这是什么鬼 ……


4. 如果 User 有 __getattr__ 方法, 调用 __getattr__ 方法，

5. 否则 抛出异常
"""
    

class User:
    """
    一个属性就是一个属性描述符的实例化对象
    在对属性描述符实例对象使用 = 赋值的时候 ==> 变为调用 __set__ 方法
    在获取属性描述符的实例对象的值得时候    ==> 变为调用 __get__ 方法
    """
    # age = IntField()  
    
    # 当 age 是非数据描述符 的时候，
    # 因为没有实现 __set__ 所以并能为其赋值从而在实例对象中创建了 age 变量
    age = NonDataField() 

    
    
# class User:
#     def __init__(self):
#         # 是可以在实例对象中使用属性描述符的，but, 直接返回了 user.__dict__['age']
#         age = IntField()  
    
user = User()
"""
因为在实例对象 user 中并不存在 age 属性
所以按照之前的认知，当使用 user.age 的时候
应该会在 user.__dict__ 中生成一个新的 age 变量
而，现在却不是这样
"""
"""
即使 user 的 __dict__ 中存在 age 属性
当使用 点 的方式访问 user 中的属性的时候
若 User.age 是数据描述符，那么依然有限访问 User.age 

而若用 user.__dict__ 来访问 age 属性则可以获取到 user 中的 age
使用 __dict__ 的方式来查找属性 ==> 不涉及属性查找过程
"""
user.age = 666
# User.age = 777
print(User.__dict__)
print(" --------------------- ")
print(user.__dict__)
print(" --------------------- ")
print(User.age)
print(user.age)

{'__module__': '__main__', '__doc__': '\n    一个属性就是一个属性描述符的实例化对象\n    在对属性描述符实例对象使用 = 赋值的时候 ==> 变为调用 __set__ 方法\n    在获取属性描述符的实例对象的值得时候    ==> 变为调用 __get__ 方法\n    ', 'age': 777, '__dict__': <attribute '__dict__' of 'User' objects>, '__weakref__': <attribute '__weakref__' of 'User' objects>}
 --------------------- 
{'age': 666}
 --------------------- 
777
666
